In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import scanpy as sc
import torch
import lineagevi as linvi
import numpy as np
import pandas as pd

In [3]:
adata_path = '/Users/lgolinelli/git/lineageVI/notebooks/data/outputs/pancreas_2025.08.17_12.43.17/adata_with_velocity.h5ad'
model_path = '/Users/lgolinelli/git/lineageVI/notebooks/data/outputs/pancreas_2025.08.17_12.43.17/vae_velocity_model.pt'

adata = sc.read_h5ad(adata_path)

model = linvi.trainer.LineageVI(
    adata,
)

model.load_state_dict(torch.load(model_path, map_location='cpu'))
model.eval()

LineageVI(
  (encoder): Encoder(
    (encoder): Sequential(
      (0): Linear(in_features=1805, out_features=128, bias=True)
      (1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (2): ReLU()
    )
    (mean_layer): Linear(in_features=128, out_features=647, bias=True)
    (logvar_layer): Linear(in_features=128, out_features=647, bias=True)
  )
  (gene_decoder): MaskedLinearDecoder(
    (linear): Linear(in_features=647, out_features=1805, bias=True)
  )
  (velocity_decoder): VelocityDecoder(
    (shared_decoder): Sequential(
      (0): Linear(in_features=647, out_features=128, bias=True)
      (1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (2): ReLU()
    )
    (gp_velocity_decoder): Linear(in_features=128, out_features=647, bias=True)
    (gene_velocity_decoder): Sequential(
      (0): Linear(in_features=128, out_features=5415, bias=True)
      (1): Softplus(beta=1.0, threshold=20.0)
    )
  )
)

In [4]:
cats = adata.obs['clusters'].cat.categories
present = adata.obs['clusters'].unique()
unused = cats.difference(present)

print("All categories:", cats)
print("Present:", present)
print("Unused:", unused)


All categories: Index(['Ductal', 'Ngn3 low EP', 'Ngn3 high EP', 'Pre-endocrine', 'Beta',
       'Alpha', 'Delta', 'Epsilon'],
      dtype='object')
Present: ['Pre-endocrine', 'Ductal', 'Alpha', 'Ngn3 high EP', 'Delta', 'Beta', 'Ngn3 low EP', 'Epsilon']
Categories (8, object): ['Ductal', 'Ngn3 low EP', 'Ngn3 high EP', 'Pre-endocrine', 'Beta', 'Alpha', 'Delta', 'Epsilon']
Unused: Index([], dtype='object')


In [5]:
adata.obs['clusters']

index
AAACCTGAGAGGGATA    Pre-endocrine
AAACCTGAGCCTTGAT           Ductal
AAACCTGAGGCAATTA            Alpha
AAACCTGCATCATCCC           Ductal
AAACCTGGTAAGTGGC     Ngn3 high EP
                        ...      
TTTGTCAAGTGACATA    Pre-endocrine
TTTGTCAAGTGTGGCA     Ngn3 high EP
TTTGTCAGTTGTTTGG           Ductal
TTTGTCATCGAATGCT            Alpha
TTTGTCATCTGTTTGT          Epsilon
Name: clusters, Length: 3696, dtype: category
Categories (8, object): ['Ductal', 'Ngn3 low EP', 'Ngn3 high EP', 'Pre-endocrine', 'Beta', 'Alpha', 'Delta', 'Epsilon']

In [6]:
df_genes, df_gps, perturbed_outputs = model.perturb_genes(
                adata=adata,
                cell_type_key='clusters',
                cell_type_to_perturb='Beta',
                genes_to_perturb=['Sntg1', 'Snhg6'],
                perturb_value=0,
                perturb_spliced=True
            )


In [7]:
df_genes, df_gps, perturbed_outputs = model.perturb_gps(
                adata=adata,
                gp_uns_key='terms',
                gps_to_perturb=['YBX1_TARGETS_DN', 'YBX1_TARGETS_UP'],
                cell_type_key='clusters',
                ctypes_to_perturb='Beta',
                perturb_value=0
            )